In [1]:
# importing required libraries
import pandas as pd
import numpy as np

In [ ]:
#relative path to data
excel = 'data/Online Retail.xlsx'

In [ ]:
df = pd.read_excel(excel)
data = df

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
df.shape

There are 25900 unique transactions with 4372 customer over 4070 unique items from 38 unique countries.

## Cleaning the DATA

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#data where description is null
df[df.isnull().Description].head()

In [ ]:
#shape of the data where description is NaN
df[df.isnull().Description].shape

we have 1454 entries where description is null so we will remove those enteries

In [ ]:
df = df[~df.isnull().Description]
df = df.reset_index(drop=True)#reindexing

deleting the rows where quantity is less than 1

In [ ]:
df[df.Quantity < 1].shape

In [ ]:
df = df[df.Quantity >=1]
df = df.reset_index(drop=True)#reindexing

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
#some invoiceNo we not number but they containde some alpha charaters getting the count of them
a = df.InvoiceNo 
a = a.unique()
count = 0
alphaInvoice = pd.Series([])
for i in a:
    try:
        float(i)
    except:
        alphaInvoice = alphaInvoice.set_value(count , i)
        count = count + 1
print(count)
print(alphaInvoice)

In [ ]:
#checking the main data for alphaInvoices
df[df.InvoiceNo.isin(alphaInvoice)]

as we can see that , the above transactions were of no use they are just populating data and corrupting the Invoice Pattern we will remove it

In [ ]:
df = df[~df.InvoiceNo.isin(alphaInvoice)]

In [ ]:
df.InvoiceNo = df.InvoiceNo.astype(float)
df.InvoiceNo.dtype

In [ ]:
df.dtypes

there are lot of customer which do not have any id , so we will generate customer id on the basics of InvoiceNo.
Also we have changed the type of the invoiceId to float64 to match it with the type of CustomerId(also it will make sense as )


but before that we will check that any customer id is equal to invoice number or not

In [ ]:
print(pd.Series(df.InvoiceNo.unique()).isin(pd.Series(df.CustomerID.unique())).sum())

In [ ]:
#total number of unique customers
df.CustomerID.nunique()

In [ ]:
#total number of unique invoice where customerId is NaN
df[df.CustomerID.isnull()].InvoiceNo.nunique()

In [ ]:
#where the customer id is NaN changing it with the ivoice number so that we get some identity for the customer
# and then the total number of unique customers should be -> 4339 + 1597(= 5936)(if no customerid(present in data) is equal to invoiceId(where customer id is NaN))
df.CustomerID =df.CustomerID.fillna(df.InvoiceNo) 
df.CustomerID.nunique()

In [ ]:
df.isnull().sum()

All NaN removed successfully Data Cleaned

In [ ]:
#final check to check data manually
def uniquePrinter(df):
    x = df.columns
    for i in x:
        print("data for {}".format(i))
        rowData = df[i].unique()
        for j in rowData:
            print(j)
            
#checked

on seeing the data it was observed that some quantites are very large and they should not be taken for prediction.
as they can be from customer who buy item in wholesale , so it is not good to use this data for retail customer and the data should be generalized

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.boxplot(df.Quantity)

In [ ]:
plt.boxplot(df[df.Quantity < 15].Quantity)

In [ ]:
df.Quantity.describe()

Taking the q3(third quartile for boxplot limit) to restrict the data for generalisation

In [ ]:
df = df[df.Quantity <= 10*1.5] 

In [ ]:
df.Quantity.describe()

Drastic changes observed : we can see the standar deviation has changed drastically this was due to the outliers , tis issue is resolved

In [ ]:
df.describe()

In [ ]:
plt.boxplot(df.UnitPrice)

In [ ]:
test_df = df.copy()
test_df[test_df.UnitPrice > 100]

here we see that majority of over priced items are actually not items they are postages or shipments so we will remove those items : the item stockcode are ('DOT','POST','M','AMAZONFEE','C2')

In [ ]:
test = test_df.StockCode == "POST" 

In [ ]:
test_df = test_df[~test]

In [ ]:
test = test_df.StockCode == "DOT" 

In [ ]:
test_df = test_df[~test]

In [ ]:
test = test_df.StockCode == "M" 

In [ ]:
test_df = test_df[~test]

In [ ]:
test = test_df.StockCode == "AMAZONFEE" 

In [ ]:
test_df = test_df[~test]

In [ ]:
test = test_df.StockCode == "C2" 

In [ ]:
test_df = test_df[~test]

In [ ]:
test_df[test_df.UnitPrice > 100]

In [ ]:
test_df.UnitPrice.describe()

we can see that the standard deviation has dropped drastically

the last thing to check is that min unit price is 0 which has description of amazon and post , also it sounds logical that no mart will sell items for free.so we will remove this.


In [ ]:
test_df = test_df[~(test_df.UnitPrice == 0)]

In [ ]:
test_df.shape

In [ ]:
df = test_df.copy()

In [ ]:
df.to_csv("./data/finalData.csv",index=False)

checking errors in stockcode and description(item)

In [ ]:
df.StockCode.nunique()

In [ ]:
df.Description.nunique()

we can see that some stock codes are not equal to the product name, we will find those items so for the prediction we will only use the description not the stockCode